In [ ]:
# Lagrangian Dynamics for Robotic Systems
# Interactive Google Colab Version

# Install required libraries
!pip install sympy numpy matplotlib ipywidgets scipy

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from sympy.utilities.lambdify import lambdify
from scipy.integrate import solve_ivp
from IPython.display import HTML, display, Image
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
import time

print("=================================================================")
print("  Lagrangian Dynamics for Robotic Systems - Interactive Version")
print("  Advanced Robotics with ROS - Module 2")
print("=================================================================")

# Functions for pretty printing
def print_section(title):
    """Print a formatted section header"""
    print("\n" + "=" * 80)
    print(f" {title}")
    print("=" * 80 + "\n")

def print_eq(name, equation):
    """Print an equation with a name"""
    print(f"{name}:")
    display(equation)
    print()

# Configure sympy for nice display
sp.init_printing(use_unicode=True)

# ================================================================
# Part 1: Simple Pendulum
# ================================================================
print_section("Part 1: Simple Pendulum")

# Define symbolic variables
theta, theta_dot, theta_ddot = sp.symbols('theta theta_dot theta_ddot', real=True)
m, g, l = sp.symbols('m g l', positive=True)
tau = sp.symbols('tau', real=True)

# Define generalized coordinates q and their derivatives
q = theta
q_dot = theta_dot
q_ddot = theta_ddot

# Step 1: Define the position of the mass
x = l * sp.sin(theta)
y = -l * sp.cos(theta)

# Step 2: Compute velocities (time derivatives of position)
x_dot = sp.diff(x, theta) * theta_dot
y_dot = sp.diff(y, theta) * theta_dot

# Step 3: Compute kinetic energy
T = (1/2) * m * (x_dot**2 + y_dot**2)
print_eq("Kinetic Energy (T)", T)

# Step 4: Compute potential energy
V = m * g * y  # Potential energy (reference at pivot)
print_eq("Potential Energy (V)", V)

# Step 5: Form the Lagrangian L = T - V
L = T - V
print_eq("Lagrangian (L = T - V)", L)

# Step 6: Compute partial derivatives for Lagrange's equations
dL_dq = sp.diff(L, q)
dL_dq_dot = sp.diff(L, q_dot)
d_dt_dL_dq_dot = sp.diff(dL_dq_dot, q) * q_dot + sp.diff(dL_dq_dot, q_dot) * q_ddot

print_eq("∂L/∂θ", dL_dq)
print_eq("∂L/∂θ̇", dL_dq_dot)
print_eq("d/dt(∂L/∂θ̇)", d_dt_dL_dq_dot)

# Step 7: Form Lagrange's equation: d/dt(∂L/∂q̇) - ∂L/∂q = Q
# where Q is the generalized force (torque)
lagrange_eq = sp.Eq(d_dt_dL_dq_dot - dL_dq, tau)
print_eq("Lagrange's Equation (with external torque tau)", lagrange_eq)

# Step 8: Solve for the acceleration (equation of motion)
eq_of_motion = sp.solve(lagrange_eq, q_ddot)[0]
print_eq("Equation of Motion (solved for theta_ddot)", eq_of_motion)

# Step 9: Put in standard form
eq_standard = sp.simplify(sp.collect(eq_of_motion, [tau, sp.sin(theta)]))
print_eq("Equation in Standard Form", eq_standard)

# Convert to a function for numerical simulation
pendulum_ode_func = lambdify((theta, theta_dot, tau, m, l, g), eq_standard)

print("Creating interactive pendulum simulation...")

def simulate_pendulum(initial_angle, initial_velocity, pendulum_length, pendulum_mass, 
                     gravity, external_torque, simulation_time, damping=0.1):
    """Simulate a pendulum with given parameters"""
    
    # Define the ODE function with damping
    def pendulum_ode(t, y):
        theta, theta_dot = y
        # Add damping term (-damping * theta_dot)
        theta_ddot = pendulum_ode_func(theta, theta_dot, external_torque, 
                                      pendulum_mass, pendulum_length, gravity) - damping * theta_dot
        return [theta_dot, theta_ddot]
    
    # Initial conditions [theta, theta_dot]
    y0 = [initial_angle, initial_velocity]
    
    # Time span
    t_span = [0, simulation_time]
    t_eval = np.linspace(0, simulation_time, 200)
    
    # Solve ODE
    sol = solve_ivp(pendulum_ode, t_span, y0, method='RK45', t_eval=t_eval)
    
    # Calculate pendulum positions for animation
    x_pos = pendulum_length * np.sin(sol.y[0])
    y_pos = -pendulum_length * np.cos(sol.y[0])
    
    return sol.t, sol.y, x_pos, y_pos

def plot_pendulum_results(t, y, x_pos, y_pos, pendulum_length):
    """Plot the pendulum simulation results"""
    
    fig = plt.figure(figsize=(12, 10))
    
    # Add axis for angle plot
    ax1 = fig.add_subplot(2, 2, 1)
    ax1.plot(t, y[0])
    ax1.set_title('Pendulum Angle')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Angle (rad)')
    ax1.grid(True)
    
    # Add axis for angular velocity plot
    ax2 = fig.add_subplot(2, 2, 2)
    ax2.plot(t, y[1])
    ax2.set_title('Pendulum Angular Velocity')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Angular Velocity (rad/s)')
    ax2.grid(True)
    
    # Add axis for pendulum animation
    ax3 = fig.add_subplot(2, 2, (3, 4))
    ax3.set_xlim(-1.2*pendulum_length, 1.2*pendulum_length)
    ax3.set_ylim(-1.2*pendulum_length, 1.2*pendulum_length)
    ax3.set_aspect('equal')
    ax3.grid(True)
    
    # Create a line for the pendulum
    line, = ax3.plot([], [], 'o-', lw=2, markersize=10)
    
    # Add time text
    time_template = 'time = %.1fs'
    time_text = ax3.text(0.05, 0.9, '', transform=ax3.transAxes)
    
    # Animation initialization function
    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line, time_text
    
    # Animation update function
    def animate(i):
        line.set_data([0, x_pos[i]], [0, y_pos[i]])
        time_text.set_text(time_template % t[i])
        return line, time_text
    
    # Create animation
    ani = FuncAnimation(fig, animate, frames=len(t),
                       init_func=init, blit=True, interval=50)
    
    plt.tight_layout()
    plt.show()
    
    return ani

# Create interactive widgets
pendulum_widgets = {
    'initial_angle': widgets.FloatSlider(value=np.pi/4, min=-np.pi, max=np.pi, step=0.1, 
                                        description='Initial Angle:', readout_format='.2f'),
    'initial_velocity': widgets.FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, 
                                           description='Initial Velocity:'),
    'pendulum_length': widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, 
                                          description='Length (m):'),
    'pendulum_mass': widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, 
                                        description='Mass (kg):'),
    'gravity': widgets.FloatSlider(value=9.81, min=0.0, max=20.0, step=0.1, 
                                  description='Gravity (m/s²):'),
    'external_torque': widgets.FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, 
                                          description='Torque (Nm):'),
    'damping': widgets.FloatSlider(value=0.1, min=0.0, max=2.0, step=0.05, 
                                  description='Damping:'),
    'simulation_time': widgets.FloatSlider(value=10.0, min=1.0, max=30.0, step=1.0, 
                                          description='Sim Time (s):')
}

# Create a button to run the simulation
simulate_button = widgets.Button(description="Run Simulation")
output_area = widgets.Output()

# Layout for widgets
controls = widgets.VBox([
    widgets.HBox([pendulum_widgets['initial_angle'], pendulum_widgets['initial_velocity']]),
    widgets.HBox([pendulum_widgets['pendulum_length'], pendulum_widgets['pendulum_mass']]),
    widgets.HBox([pendulum_widgets['gravity'], pendulum_widgets['external_torque']]),
    widgets.HBox([pendulum_widgets['damping'], pendulum_widgets['simulation_time']]),
    simulate_button
])

# Run the simulation when the button is clicked
def on_simulate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Running simulation...")
        
        # Get parameter values from widgets
        initial_angle = pendulum_widgets['initial_angle'].value
        initial_velocity = pendulum_widgets['initial_velocity'].value
        pendulum_length = pendulum_widgets['pendulum_length'].value
        pendulum_mass = pendulum_widgets['pendulum_mass'].value
        gravity = pendulum_widgets['gravity'].value
        external_torque = pendulum_widgets['external_torque'].value
        simulation_time = pendulum_widgets['simulation_time'].value
        damping = pendulum_widgets['damping'].value
        
        # Run simulation
        t, y, x_pos, y_pos = simulate_pendulum(
            initial_angle, initial_velocity, pendulum_length, pendulum_mass, 
            gravity, external_torque, simulation_time, damping
        )
        
        # Plot results
        ani = plot_pendulum_results(t, y, x_pos, y_pos, pendulum_length)

simulate_button.on_click(on_simulate_button_clicked)

# Display the interactive elements
display(controls, output_area)

# ================================================================
# Part 2: Double Pendulum
# ================================================================
print_section("Part 2: Double Pendulum")

# Define symbolic variables
theta1, theta2 = sp.symbols('theta1 theta2', real=True)
theta1_dot, theta2_dot = sp.symbols('theta1_dot theta2_dot', real=True)
theta1_ddot, theta2_ddot = sp.symbols('theta1_ddot theta2_ddot', real=True)
m1, m2, l1, l2, g = sp.symbols('m1 m2 l1 l2 g', positive=True)
tau1, tau2 = sp.symbols('tau1 tau2', real=True)

# Step 1: Define the positions of the masses
# Position of mass 1
x1 = l1 * sp.sin(theta1)
y1 = -l1 * sp.cos(theta1)

# Position of mass 2
x2 = l1 * sp.sin(theta1) + l2 * sp.sin(theta2)
y2 = -l1 * sp.cos(theta1) - l2 * sp.cos(theta2)

print_eq("Position of Mass 1 (x1, y1)", [x1, y1])
print_eq("Position of Mass 2 (x2, y2)", [x2, y2])

# Step 2: Compute velocities
# Velocities of mass 1
x1_dot = sp.diff(x1, theta1) * theta1_dot
y1_dot = sp.diff(y1, theta1) * theta1_dot

# Velocities of mass 2
x2_dot = sp.diff(x2, theta1) * theta1_dot + sp.diff(x2, theta2) * theta2_dot
y2_dot = sp.diff(y2, theta1) * theta1_dot + sp.diff(y2, theta2) * theta2_dot

print_eq("Velocity of Mass 1 (ẋ1, ẏ1)", [x1_dot, y1_dot])
print_eq("Velocity of Mass 2 (ẋ2, ẏ2)", [x2_dot, y2_dot])

# Step 3: Compute kinetic energy
T1 = (1/2) * m1 * (x1_dot**2 + y1_dot**2)
T2 = (1/2) * m2 * (x2_dot**2 + y2_dot**2)
T = T1 + T2
print_eq("Kinetic Energy (T)", sp.simplify(T))

# Step 4: Compute potential energy
V1 = m1 * g * y1
V2 = m2 * g * y2
V = V1 + V2
print_eq("Potential Energy (V)", sp.simplify(V))

# Step 5: Form the Lagrangian L = T - V
L = T - V
print("Lagrangian (L = T - V): Expression too complex to display in full form")

# Simulate Double Pendulum
print("Creating interactive double pendulum simulation...")

def double_pendulum_ode(t, y, m1, m2, l1, l2, g):
    """
    ODE for double pendulum motion
    
    Parameters:
        t: time (not used, required by solver)
        y: state vector [theta1, theta2, omega1, omega2]
        m1, m2: masses
        l1, l2: lengths
        g: gravity
    """
    theta1, theta2, omega1, omega2 = y
    
    # Calculate accelerations using analytical solution
    # These equations are derived from Lagrangian mechanics
    # but explicitly written out to avoid symbolic computation overhead
    
    delta = theta2 - theta1
    den1 = (m1 + m2) * l1
    den2 = m2 * l2 * np.cos(delta)
    
    # Denominator of the matrix equation
    denom = l1 * l2 * (m1 + m2 * np.sin(delta)**2)
    
    # Right-hand side terms
    f1 = -m2 * l2 * omega2**2 * np.sin(delta) - (m1 + m2) * g * np.sin(theta1)
    f2 = l1 * omega1**2 * np.sin(delta) - g * np.sin(theta2)
    
    # Accelerations
    alpha1 = (l2 * f1 - m2 * l2 * np.cos(delta) * f2) / denom
    alpha2 = ((m1 + m2) * l1 * np.cos(delta) * f1 - (m1 + m2) * l1 * f2) / denom
    
    return [omega1, omega2, alpha1, alpha2]

def simulate_double_pendulum(theta1_0, theta2_0, omega1_0, omega2_0, m1, m2, l1, l2, g, duration, damping=0.0):
    """Simulate a double pendulum system"""
    
    # Initial state [theta1, theta2, omega1, omega2]
    y0 = [theta1_0, theta2_0, omega1_0, omega2_0]
    
    # Add damping by modifying the ODE function
    def ode_with_damping(t, y):
        dydt = double_pendulum_ode(t, y, m1, m2, l1, l2, g)
        # Add damping to angular velocities
        dydt[2] -= damping * y[2]  # Damping for omega1
        dydt[3] -= damping * y[3]  # Damping for omega2
        return dydt
    
    # Time span
    t_span = [0, duration]
    t_eval = np.linspace(0, duration, 500)
    
    # Solve ODE
    sol = solve_ivp(ode_with_damping, t_span, y0, method='RK45', t_eval=t_eval, rtol=1e-6)
    
    # Extract results
    theta1 = sol.y[0]
    theta2 = sol.y[1]
    
    # Calculate positions
    x1 = l1 * np.sin(theta1)
    y1 = -l1 * np.cos(theta1)
    x2 = x1 + l2 * np.sin(theta2)
    y2 = y1 - l2 * np.cos(theta2)
    
    return sol.t, theta1, theta2, sol.y[2], sol.y[3], x1, y1, x2, y2

def animate_double_pendulum(t, x1, y1, x2, y2, l1, l2):
    """Animate the double pendulum simulation"""
    
    # Set up the figure
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-1.2 * (l1 + l2), 1.2 * (l1 + l2))
    ax.set_ylim(-1.2 * (l1 + l2), 1.2 * (l1 + l2))
    ax.set_aspect('equal')
    ax.grid()
    
    # Lines representing the pendulum rods
    line1, = ax.plot([], [], 'o-', lw=2, markersize=10, color='blue')
    line2, = ax.plot([], [], 'o-', lw=2, markersize=10, color='red')
    
    # Time text
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    # Trails for the pendulum bobs
    trail2, = ax.plot([], [], '-', lw=1, color='red', alpha=0.3)
    trail_x = []
    trail_y = []
    max_trail_length = 100  # Adjust as needed
    
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        trail2.set_data([], [])
        return line1, line2, time_text, trail2
    
    def animate(i):
        line1.set_data([0, x1[i]], [0, y1[i]])
        line2.set_data([x1[i], x2[i]], [y1[i], y2[i]])
        
        # Update trail
        trail_x.append(x2[i])
        trail_y.append(y2[i])
        if len(trail_x) > max_trail_length:
            trail_x.pop(0)
            trail_y.pop(0)
        trail2.set_data(trail_x, trail_y)
        
        time_text.set_text(time_template % t[i])
        return line1, line2, time_text, trail2
    
    # Create animation
    ani = FuncAnimation(fig, animate, frames=range(0, len(t), 2),  # Skip frames for speed
                      init_func=init, blit=True, interval=50)
    
    plt.title('Double Pendulum Animation')
    plt.tight_layout()
    plt.show()
    
    return ani

# Create interactive widgets for double pendulum
double_pendulum_widgets = {
    'theta1_0': widgets.FloatSlider(value=np.pi/2, min=-np.pi, max=np.pi, step=0.1, 
                                   description='θ₁ initial:'),
    'theta2_0': widgets.FloatSlider(value=np.pi/4, min=-np.pi, max=np.pi, step=0.1, 
                                   description='θ₂ initial:'),
    'omega1_0': widgets.FloatSlider(value=0.0, min=-3.0, max=3.0, step=0.1, 
                                   description='ω₁ initial:'),
    'omega2_0': widgets.FloatSlider(value=0.0, min=-3.0, max=3.0, step=0.1, 
                                   description='ω₂ initial:'),
    'm1': widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, 
                             description='Mass 1 (kg):'),
    'm2': widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, 
                             description='Mass 2 (kg):'),
    'l1': widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, 
                             description='Length 1 (m):'),
    'l2': widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, 
                             description='Length 2 (m):'),
    'g': widgets.FloatSlider(value=9.81, min=0.0, max=20.0, step=0.1, 
                           description='Gravity (m/s²):'),
    'damping': widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, 
                                  description='Damping:'),
    'duration': widgets.FloatSlider(value=20.0, min=5.0, max=60.0, step=5.0, 
                                   description='Duration (s):')
}

# Create a button to run the simulation
double_pendulum_button = widgets.Button(description="Run Double Pendulum Simulation")
double_pendulum_output = widgets.Output()

# Layout for widgets
double_pendulum_controls = widgets.VBox([
    widgets.HBox([double_pendulum_widgets['theta1_0'], double_pendulum_widgets['theta2_0']]),
    widgets.HBox([double_pendulum_widgets['omega1_0'], double_pendulum_widgets['omega2_0']]),
    widgets.HBox([double_pendulum_widgets['m1'], double_pendulum_widgets['m2']]),
    widgets.HBox([double_pendulum_widgets['l1'], double_pendulum_widgets['l2']]),
    widgets.HBox([double_pendulum_widgets['g'], double_pendulum_widgets['damping']]),
    widgets.HBox([double_pendulum_widgets['duration']]),
    double_pendulum_button
])

# Function to run when button is clicked
def on_double_pendulum_button_clicked(b):
    with double_pendulum_output:
        double_pendulum_output.clear_output()
        print("Running double pendulum simulation...")
        
        # Get parameter values
        theta1_0 = double_pendulum_widgets['theta1_0'].value
        theta2_0 = double_pendulum_widgets['theta2_0'].value
        omega1_0 = double_pendulum_widgets['omega1_0'].value
        omega2_0 = double_pendulum_widgets['omega2_0'].value
        m1 = double_pendulum_widgets['m1'].value
        m2 = double_pendulum_widgets['m2'].value
        l1 = double_pendulum_widgets['l1'].value
        l2 = double_pendulum_widgets['l2'].value
        g = double_pendulum_widgets['g'].value
        damping = double_pendulum_widgets['damping'].value
        duration = double_pendulum_widgets['duration'].value
        
        # Run simulation
        t, theta1, theta2, omega1, omega2, x1, y1, x2, y2 = simulate_double_pendulum(
            theta1_0, theta2_0, omega1_0, omega2_0, m1, m2, l1, l2, g, duration, damping
        )
        
        # Plot and animate
        ani = animate_double_pendulum(t, x1, y1, x2, y2, l1, l2)
        
        # Plot energy over time
        # Kinetic energy
        KE1 = 0.5 * m1 * l1**2 * omega1**2
        KE2 = 0.5 * m2 * (l1**2 * omega1**2 + l2**2 * omega2**2 + 
                          2 * l1 * l2 * omega1 * omega2 * np.cos(theta1 - theta2))
        
        # Potential energy
        PE1 = m1 * g * l1 * (1 - np.cos(theta1))
        PE2 = m2 * g * (l1 * (1 - np.cos(theta1)) + l2 * (1 - np.cos(theta2)))
        
        # Total energy
        E = KE1 + KE2 + PE1 + PE2
        
        plt.figure(figsize=(10, 6))
        plt.plot(t, KE1 + KE2, label='Kinetic Energy')
        plt.plot(t, PE1 + PE2, label='Potential Energy')
        plt.plot(t, E, label='Total Energy')
        plt.xlabel('Time (s)')
        plt.ylabel('Energy (J)')
        plt.title('Energy in Double Pendulum System')
        plt.legend()
        plt.grid(True)
        plt.show()

double_pendulum_button.on_click(on_double_pendulum_button_clicked)

# Display the double pendulum controls
display(double_pendulum_controls, double_pendulum_output)

# ================================================================
# Part 3: Differential Drive Robot
# ================================================================
print_section("Part 3: Differential Drive Robot")

print("Creating interactive differential drive robot simulation...")

def differential_drive_dynamics(t, state, m, I, r, L, tau_r, tau_l, mu):
    """
    Dynamics of a differential drive robot
    
    Parameters:
        t: time (not used, required by solver)
        state: state vector [x, y, theta, phi_r, phi_l, x_dot, y_dot, theta_dot, phi_r_dot, phi_l_dot]
        m: robot mass
        I: robot moment of inertia
        r: wheel radius
        L: distance between wheels
        tau_r, tau_l: right and left wheel torques
        mu: friction coefficient
    """
    # Unpack state
    x, y, theta, phi_r, phi_l, x_dot, y_dot, theta_dot, phi_r_dot, phi_l_dot = state
    
    # Simplified dynamics based on the Lagrangian derivation
    # These are based on the non-holonomic constraint equations
    
    # Wheel accelerations with friction
    phi_r_ddot = (tau_r - mu * phi_r_dot) / I
    phi_l_ddot = (tau_l - mu * phi_l_dot) / I
    
    # Robot accelerations from wheel accelerations
    # Based on the kinematic relationships
    theta_ddot = (r / L) * (phi_r_ddot - phi_l_ddot)
    v_ddot = (r / 2) * (phi_r_ddot + phi_l_ddot)
    
    # Cartesian accelerations
    x_ddot = v_ddot * np.cos(theta) - y_dot * theta_dot
    y_ddot = v_ddot * np.sin(theta) + x_dot * theta_dot
    
    # Return state derivatives
    return [x_dot, y_dot, theta_dot, phi_r_dot, phi_l_dot, 
            x_ddot, y_ddot, theta_ddot, phi_r_ddot, phi_l_ddot]

def simulate_diff_drive(duration, m, I, r, L, tau_r, tau_l, mu):
    """Simulate a differential drive robot"""
    
    # Initial state [x, y, theta, phi_r, phi_l, x_dot, y_dot, theta_dot, phi_r_dot, phi_l_dot]
    initial_state = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    # Time span
    t_span = [0, duration]
    t_eval = np.linspace(0, duration, 500)
    
    # Create a function that includes the parameters
    def system(t, state):
        return differential_drive_dynamics(t, state, m, I, r, L, tau_r, tau_l, mu)
    
    # Solve ODE
    sol = solve_ivp(system, t_span, initial_state, method='RK45', t_eval=t_eval)
    
    # Extract results
    x = sol.y[0]
    y = sol.y[1]
    theta = sol.y[2]
    phi_r = sol.y[3]
    phi_l = sol.y[4]
    
    return sol.t, x, y, theta, phi_r, phi_l

def plot_diff_drive_results(t, x, y, theta):
    """Plot the results of a differential drive robot simulation"""
    
    # Create figure with 3 subplots
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot x position
    axs[0, 0].plot(t, x)
    axs[0, 0].set_xlabel('Time (s)')
    axs[0, 0].set_ylabel('X Position (m)')
    axs[0, 0].set_title('X Position vs Time')
    axs[0, 0].grid(True)
    
    # Plot y position
    axs[0, 1].plot(t, y)
    axs[0, 1].set_xlabel('Time (s)')
    axs[0, 1].set_ylabel('Y Position (m)')
    axs[0, 1].set_title('Y Position vs Time')
    axs[0, 1].grid(True)
    
    # Plot orientation
    axs[1, 0].plot(t, theta)
    axs[1, 0].set_xlabel('Time (s)')
    axs[1, 0].set_ylabel('Orientation (rad)')
    axs[1, 0].set_title('Orientation vs Time')
    axs[1, 0].grid(True)
    
    # Plot trajectory in 2D
    axs[1, 1].plot(x, y)
    axs[1, 1].set_xlabel('X Position (m)')
    axs[1, 1].set_ylabel('Y Position (m)')
    axs[1, 1].set_title('Robot Trajectory')
    axs[1, 1].set_aspect('equal')
    axs[1, 1].grid(True)
    
    # Add robot orientation markers
    skip = max(1, len(t) // 20)  # Show only a few orientation markers
    for i in range(0, len(t), skip):
        # Draw a small line indicating robot orientation
        arrow_length = 0.2
        dx = arrow_length * np.cos(theta[i])
        dy = arrow_length * np.sin(theta[i])
        axs[1, 1].arrow(x[i], y[i], dx, dy, head_width=0.05, head_length=0.1, fc='red', ec='red')
    
    plt.tight_layout()
    plt.show()
    
    return fig

def animate_diff_drive(t, x, y, theta, radius, wheel_base):
    """Create an animation of the differential drive robot"""
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Determine the limits for the plot based on robot trajectory
    margin = 1.0
    x_min, x_max = min(x) - margin, max(x) + margin
    y_min, y_max = min(y) - margin, max(y) + margin
    
    # Set equal aspect ratio to ensure circles look like circles
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_aspect('equal')
    ax.grid(True)
    
    # Create artists for robot visualization
    robot_body, = ax.plot([], [], 'bo', markersize=15)  # Robot center
    robot_direction, = ax.plot([], [], 'r-', linewidth=2)  # Direction indicator
    left_wheel, = ax.plot([], [], 'ko', markersize=5)  # Left wheel
    right_wheel, = ax.plot([], [], 'ko', markersize=5)  # Right wheel
    path, = ax.plot([], [], 'g-', linewidth=1, alpha=0.7)  # Path trace
    
    # Time indicator
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes)
    
    # Arrays to store the path for animation
    path_x = []
    path_y = []
    
    def init():
        """Initialize the animation"""
        robot_body.set_data([], [])
        robot_direction.set_data([], [])
        left_wheel.set_data([], [])
        right_wheel.set_data([], [])
        path.set_data([], [])
        time_text.set_text('')
        return robot_body, robot_direction, left_wheel, right_wheel, path, time_text
    
    def animate(i):
        """Update the animation for frame i"""
        # Current position and orientation
        xi, yi, thetai = x[i], y[i], theta[i]
        
        # Robot direction indicator (forward direction)
        direction_length = 0.3
        x_dir = xi + direction_length * np.cos(thetai)
        y_dir = yi + direction_length * np.sin(thetai)
        robot_direction.set_data([xi, x_dir], [yi, y_dir])
        
        # Robot body
        robot_body.set_data([xi], [yi])
        
        # Wheel positions
        # Calculate wheel positions perpendicular to robot direction
        wheel_offset = wheel_base / 2
        left_x = xi - wheel_offset * np.sin(thetai)
        left_y = yi + wheel_offset * np.cos(thetai)
        right_x = xi + wheel_offset * np.sin(thetai)
        right_y = yi - wheel_offset * np.cos(thetai)
        
        left_wheel.set_data([left_x], [left_y])
        right_wheel.set_data([right_x], [right_y])
        
        # Update path
        path_x.append(xi)
        path_y.append(yi)
        path.set_data(path_x, path_y)
        
        # Update time indicator
        time_text.set_text(time_template % t[i])
        
        return robot_body, robot_direction, left_wheel, right_wheel, path, time_text
    
    # Create animation
    ani = FuncAnimation(fig, animate, frames=range(0, len(t), 5),  # Skip frames for speed
                      init_func=init, blit=True, interval=50)
    
    plt.title('Differential Drive Robot Animation')
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.tight_layout()
    plt.show()
    
    return ani

# Create interactive widgets for differential drive robot
diff_drive_widgets = {
    'mass': widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, 
                               description='Mass (kg):'),
    'inertia': widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, 
                                  description='Inertia (kg·m²):'),
    'wheel_radius': widgets.FloatSlider(value=0.1, min=0.05, max=0.3, step=0.01, 
                                       description='Wheel Radius (m):'),
    'wheel_base': widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, 
                                     description='Wheel Base (m):'),
    'tau_right': widgets.FloatSlider(value=0.5, min=-2.0, max=2.0, step=0.1, 
                                    description='Right Torque (Nm):'),
    'tau_left': widgets.FloatSlider(value=0.3, min=-2.0, max=2.0, step=0.1, 
                                   description='Left Torque (Nm):'),
    'friction': widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05, 
                                   description='Friction:'),
    'duration': widgets.FloatSlider(value=10.0, min=1.0, max=30.0, step=1.0, 
                                   description='Duration (s):')
}

# Create a button to run the simulation
diff_drive_button = widgets.Button(description="Run Differential Drive Simulation")
diff_drive_output = widgets.Output()

# Layout for widgets
diff_drive_controls = widgets.VBox([
    widgets.HBox([diff_drive_widgets['mass'], diff_drive_widgets['inertia']]),
    widgets.HBox([diff_drive_widgets['wheel_radius'], diff_drive_widgets['wheel_base']]),
    widgets.HBox([diff_drive_widgets['tau_right'], diff_drive_widgets['tau_left']]),
    widgets.HBox([diff_drive_widgets['friction'], diff_drive_widgets['duration']]),
    diff_drive_button
])

# Function to run when button is clicked
def on_diff_drive_button_clicked(b):
    with diff_drive_output:
        diff_drive_output.clear_output()
        print("Running differential drive robot simulation...")
        
        # Get parameter values
        m = diff_drive_widgets['mass'].value
        I = diff_drive_widgets['inertia'].value
        r = diff_drive_widgets['wheel_radius'].value
        L = diff_drive_widgets['wheel_base'].value
        tau_r = diff_drive_widgets['tau_right'].value
        tau_l = diff_drive_widgets['tau_left'].value
        mu = diff_drive_widgets['friction'].value
        duration = diff_drive_widgets['duration'].value
        
        # Run simulation
        t, x, y, theta, phi_r, phi_l = simulate_diff_drive(
            duration, m, I, r, L, tau_r, tau_l, mu
        )
        
        # Plot results
        plot_diff_drive_results(t, x, y, theta)
        
        # Animate robot
        ani = animate_diff_drive(t, x, y, theta, r, L)

diff_drive_button.on_click(on_diff_drive_button_clicked)

# Display the differential drive controls
display(diff_drive_controls, diff_drive_output)

# ================================================================
# Part 4: Energy Analysis
# ================================================================
print_section("Part 4: Energy Analysis")

print("""
Energy analysis is a powerful tool for understanding robotic systems. The Lagrangian 
approach is fundamentally based on energy principles, analyzing the system in terms 
of kinetic and potential energies rather than forces and accelerations.

In this section, we explore how to analyze energy in different robotic systems and 
what insights we can gain from energy-based perspectives.
""")

# Create interactive widgets for energy analysis demo
energy_widgets = {
    'system': widgets.Dropdown(
        options=[
            ('Simple Pendulum', 'pendulum'), 
            ('Double Pendulum', 'double'), 
            ('Differential Drive', 'diffdrive')
        ],
        value='pendulum',
        description='System:'
    ),
    # More widgets will be created dynamically based on the selected system
}

energy_button = widgets.Button(description="Analyze Energy")
energy_output = widgets.Output()

# Dynamic widget creation based on selection
def on_system_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Clear any existing widgets after the system dropdown
        children = list(energy_controls.children)
        energy_controls.children = [children[0]]  # Keep only the system dropdown
        
        # Create appropriate widgets based on selection
        if change['new'] == 'pendulum':
            # Widgets for simple pendulum
            energy_widgets['initial_angle'] = widgets.FloatSlider(
                value=np.pi/2, min=0, max=np.pi, step=0.1, 
                description='Initial Angle:'
            )
            energy_widgets['pendulum_length'] = widgets.FloatSlider(
                value=1.0, min=0.1, max=3.0, step=0.1, 
                description='Length (m):'
            )
            energy_widgets['damping'] = widgets.FloatSlider(
                value=0.1, min=0.0, max=1.0, step=0.05, 
                description='Damping:'
            )
            
        elif change['new'] == 'double':
            # Widgets for double pendulum
            energy_widgets['theta1_0'] = widgets.FloatSlider(
                value=np.pi/2, min=0, max=np.pi, step=0.1, 
                description='θ₁ initial:'
            )
            energy_widgets['theta2_0'] = widgets.FloatSlider(
                value=np.pi/4, min=0, max=np.pi, step=0.1, 
                description='θ₂ initial:'
            )
            energy_widgets['damping'] = widgets.FloatSlider(
                value=0.05, min=0.0, max=0.5, step=0.01, 
                description='Damping:'
            )
            
        elif change['new'] == 'diffdrive':
            # Widgets for differential drive
            energy_widgets['tau_ratio'] = widgets.FloatSlider(
                value=1.0, min=0.1, max=2.0, step=0.1, 
                description='τᵣ/τₗ ratio:'
            )
            energy_widgets['tau_magnitude'] = widgets.FloatSlider(
                value=0.5, min=0.1, max=2.0, step=0.1, 
                description='Torque magnitude:'
            )
            energy_widgets['friction'] = widgets.FloatSlider(
                value=0.1, min=0.0, max=0.5, step=0.05, 
                description='Friction:'
            )
        
        # Add the new widgets to the control panel
        new_children = [children[0]]
        for key in energy_widgets.keys():
            if key != 'system':
                new_children.append(energy_widgets[key])
        new_children.append(energy_button)
        energy_controls.children = tuple(new_children)

# Register the callback for dropdown changes
energy_widgets['system'].observe(on_system_change, names='value')

# Energy analysis functions
def analyze_pendulum_energy(initial_angle, length, damping=0.1):
    """Analyze energy in a simple pendulum system"""
    # Parameters
    m = 1.0  # kg
    g = 9.81  # m/s²
    
    # Simulate pendulum
    def pendulum_ode(t, y):
        theta, theta_dot = y
        # Equation of motion with damping
        theta_ddot = -g/length * np.sin(theta) - damping * theta_dot
        return [theta_dot, theta_ddot]
    
    # Initial conditions
    y0 = [initial_angle, 0.0]
    
    # Time span
    t_span = [0, 20]
    t_eval = np.linspace(0, 20, 500)
    
    # Solve ODE
    sol = solve_ivp(pendulum_ode, t_span, y0, method='RK45', t_eval=t_eval)
    
    # Extract results
    t = sol.t
    theta = sol.y[0]
    omega = sol.y[1]
    
    # Calculate energies
    KE = 0.5 * m * length**2 * omega**2  # Kinetic energy
    PE = m * g * length * (1 - np.cos(theta))  # Potential energy
    TE = KE + PE  # Total energy
    
    # Calculate power dissipation due to damping
    power_loss = m * length**2 * damping * omega**2
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    
    # Energy plot
    ax1.plot(t, KE, label='Kinetic Energy')
    ax1.plot(t, PE, label='Potential Energy')
    ax1.plot(t, TE, label='Total Energy')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Energy (J)')
    ax1.set_title('Energy Analysis of Simple Pendulum')
    ax1.legend()
    ax1.grid(True)
    
    # Power loss plot
    ax2.plot(t, power_loss)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Power Loss (W)')
    ax2.set_title('Power Dissipation Due to Damping')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print key insights
    print("\nEnergy Analysis Insights:")
    print(f"Initial total energy: {TE[0]:.3f} J")
    print(f"Final total energy: {TE[-1]:.3f} J")
    print(f"Energy dissipated: {TE[0] - TE[-1]:.3f} J")
    print(f"Average power dissipation: {np.mean(power_loss):.3f} W")
    
    # Phase space plot
    plt.figure(figsize=(8, 6))
    plt.plot(theta, omega)
    plt.xlabel('Angle (rad)')
    plt.ylabel('Angular Velocity (rad/s)')
    plt.title('Phase Space Trajectory')
    plt.grid(True)
    plt.show()

def analyze_double_pendulum_energy(theta1_0, theta2_0, damping=0.05):
    """Analyze energy in a double pendulum system"""
    # Parameters
    m1 = m2 = 1.0  # kg
    l1 = l2 = 1.0  # m
    g = 9.81  # m/s²
    
    # Simulate using the previously defined function
    t, theta1, theta2, omega1, omega2, x1, y1, x2, y2 = simulate_double_pendulum(
        theta1_0, theta2_0, 0.0, 0.0, m1, m2, l1, l2, g, 20.0, damping
    )
    
    # Calculate energies
    # Kinetic energy
    KE1 = 0.5 * m1 * l1**2 * omega1**2
    KE2 = 0.5 * m2 * (l1**2 * omega1**2 + l2**2 * omega2**2 + 
                      2 * l1 * l2 * omega1 * omega2 * np.cos(theta1 - theta2))
    KE = KE1 + KE2
    
    # Potential energy
    PE1 = m1 * g * l1 * (1 - np.cos(theta1))
    PE2 = m2 * g * (l1 * (1 - np.cos(theta1)) + l2 * (1 - np.cos(theta2)))
    PE = PE1 + PE2
    
    # Total energy
    TE = KE + PE
    
    # Calculate power dissipation (approximate)
    power_loss = damping * (m1 * l1**2 * omega1**2 + m2 * l2**2 * omega2**2)
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    
    # Energy plot
    ax1.plot(t, KE, label='Kinetic Energy')
    ax1.plot(t, PE, label='Potential Energy')
    ax1.plot(t, TE, label='Total Energy')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Energy (J)')
    ax1.set_title('Energy Analysis of Double Pendulum')
    ax1.legend()
    ax1.grid(True)
    
    # Power loss plot
    ax2.plot(t, power_loss)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Power Loss (W)')
    ax2.set_title('Power Dissipation Due to Damping')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print key insights
    print("\nEnergy Analysis Insights:")
    print(f"Initial total energy: {TE[0]:.3f} J")
    print(f"Final total energy: {TE[-1]:.3f} J")
    print(f"Energy dissipated: {TE[0] - TE[-1]:.3f} J")
    print(f"Average power dissipation: {np.mean(power_loss):.3f} W")
    
    # Phase space plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    ax1.plot(theta1, omega1)
    ax1.set_xlabel('θ₁ (rad)')
    ax1.set_ylabel('ω₁ (rad/s)')
    ax1.set_title('Phase Space for First Pendulum')
    ax1.grid(True)
    
    ax2.plot(theta2, omega2)
    ax2.set_xlabel('θ₂ (rad)')
    ax2.set_ylabel('ω₂ (rad/s)')
    ax2.set_title('Phase Space for Second Pendulum')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

def analyze_diff_drive_energy(tau_ratio, tau_magnitude, friction):
    """Analyze energy in a differential drive robot system"""
    # Parameters
    m = 1.0  # kg
    I = 0.1  # kg·m²
    r = 0.1  # m
    L = 0.5  # m
    tau_r = tau_magnitude
    tau_l = tau_magnitude / tau_ratio
    mu = friction
    
    # Simulate differential drive
    t, x, y, theta, phi_r, phi_l = simulate_diff_drive(
        20.0, m, I, r, L, tau_r, tau_l, mu
    )
    
    # Calculate velocities (using central differences)
    dt = t[1] - t[0]
    x_dot = np.gradient(x, dt)
    y_dot = np.gradient(y, dt)
    theta_dot = np.gradient(theta, dt)
    phi_r_dot = np.gradient(phi_r, dt)
    phi_l_dot = np.gradient(phi_l, dt)
    
    # Calculate energies
    # Kinetic energy of translation
    KE_trans = 0.5 * m * (x_dot**2 + y_dot**2)
    
    # Kinetic energy of rotation
    KE_rot = 0.5 * I * theta_dot**2
    
    # Kinetic energy of wheels
    wheel_inertia = 0.5 * I  # Simplified wheel inertia
    KE_wheels = 0.5 * wheel_inertia * (phi_r_dot**2 + phi_l_dot**2)
    
    # Total kinetic energy
    KE_total = KE_trans + KE_rot + KE_wheels
    
    # Power input from motors
    power_right = tau_r * phi_r_dot
    power_left = tau_l * phi_l_dot
    power_input = power_right + power_left
    
    # Power loss due to friction
    power_loss = mu * (phi_r_dot**2 + phi_l_dot**2)
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    
    # Energy plot
    ax1.plot(t, KE_trans, label='Translational KE')
    ax1.plot(t, KE_rot, label='Rotational KE')
    ax1.plot(t, KE_wheels, label='Wheel KE')
    ax1.plot(t, KE_total, label='Total KE')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Energy (J)')
    ax1.set_title('Energy Analysis of Differential Drive Robot')
    ax1.legend()
    ax1.grid(True)
    
    # Power plot
    ax2.plot(t, power_input, label='Power Input')
    ax2.plot(t, power_loss, label='Power Loss')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Power (W)')
    ax2.set_title('Power Analysis')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print key insights
    print("\nEnergy Analysis Insights:")
    print(f"Final kinetic energy: {KE_total[-1]:.3f} J")
    print(f"Energy efficiency: {100 * KE_total[-1] / np.trapz(power_input, t):.1f}%")
    print(f"Average power input: {np.mean(power_input):.3f} W")
    print(f"Average power loss: {np.mean(power_loss):.3f} W")
    
    # Plot robot trajectory with energy indicators
    plt.figure(figsize=(8, 6))
    
    # Plot trajectory
    plt.plot(x, y, 'b-')
    
    # Color map for energy levels
    cmap = plt.cm.jet
    norm = plt.Normalize(KE_total.min(), KE_total.max())
    
    # Plot energy-colored points along trajectory
    skip = max(1, len(t) // 50)  # Show only a subset of points
    for i in range(0, len(t), skip):
        plt.plot(x[i], y[i], 'o', color=cmap(norm(KE_total[i])), markersize=8)
    
    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm)
    cbar.set_label('Kinetic Energy (J)')
    
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.title('Robot Trajectory Colored by Energy')
    plt.axis('equal')
    plt.grid(True)
    plt.show()

# Function to run when energy analysis button is clicked
def on_energy_button_clicked(b):
    with energy_output:
        energy_output.clear_output()
        system = energy_widgets['system'].value
        
        if system == 'pendulum':
            initial_angle = energy_widgets['initial_angle'].value
            length = energy_widgets['pendulum_length'].value
            damping = energy_widgets['damping'].value
            print(f"Analyzing pendulum energy (angle={initial_angle:.2f}, length={length:.2f}m, damping={damping:.2f})...")
            analyze_pendulum_energy(initial_angle, length, damping)
            
        elif system == 'double':
            theta1_0 = energy_widgets['theta1_0'].value
            theta2_0 = energy_widgets['theta2_0'].value
            damping = energy_widgets['damping'].value
            print(f"Analyzing double pendulum energy (θ₁={theta1_0:.2f}, θ₂={theta2_0:.2f}, damping={damping:.2f})...")
            analyze_double_pendulum_energy(theta1_0, theta2_0, damping)
            
        elif system == 'diffdrive':
            tau_ratio = energy_widgets['tau_ratio'].value
            tau_magnitude = energy_widgets['tau_magnitude'].value
            friction = energy_widgets['friction'].value
            print(f"Analyzing differential drive energy (τ ratio={tau_ratio:.2f}, magnitude={tau_magnitude:.2f}, friction={friction:.2f})...")
            analyze_diff_drive_energy(tau_ratio, tau_magnitude, friction)

energy_button.on_click(on_energy_button_clicked)

# Initial setup for energy analysis controls
energy_controls = widgets.VBox([energy_widgets['system'], energy_button])

# Trigger initial widget creation
on_system_change({'type': 'change', 'name': 'value', 'new': 'pendulum'})

# Display energy analysis controls
display(energy_controls, energy_output)

# ================================================================
# Conclusion
# ================================================================
print_section("Conclusion")

print("""
In this notebook, we've explored Lagrangian dynamics for robotic systems through three examples:

1. **Simple Pendulum**: A fundamental system that demonstrates the core principles of 
   the Lagrangian approach. We derived the equations of motion from first principles and 
   simulated the system's behavior.

2. **Double Pendulum**: A more complex system that shows how the Lagrangian approach 
   handles multi-body dynamics. We saw how the equations become more complex but follow
   the same fundamental structure.

3. **Differential Drive Robot**: A practical mobile robotics example that incorporates
   non-holonomic constraints. We examined how wheel torques translate to robot motion
   and how to model the system's dynamics.

The Lagrangian approach provides a unified framework for analyzing the dynamics of 
diverse robotic systems. By focusing on energies rather than forces, it offers both 
theoretical insights and practical tools for simulation and control design.

Key takeaways:
- Lagrangian mechanics provides a systematic approach to deriving equations of motion
- Energy analysis offers valuable insights into system behavior and efficiency
- Simulation allows us to explore dynamic behavior under various conditions
- Understanding dynamics is fundamental to effective robot control and planning

These principles will be critical as we progress through the Advanced Robotics with ROS
course, particularly when implementing controllers and planning algorithms for mobile robots.
""")

print("\nThank you for exploring Lagrangian dynamics for robotic systems!")